In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [2]:
from sklearn.metrics import f1_score, precision_recall_curve


In [44]:
subsets = ['validate','test']
threshold = 0.5

for subset in subsets: 
    result_file = 'MLP-{}-results.txt'.format(subset)
    fp = open(result_file)
    lines = fp.readlines() 
    lines = [x.strip() for x in lines]
    fp.close()
    print(lines[:10])
    
    y_true = []
    y_pred = [] 
    y_prob = [] 
    
    covid_ids = [] 
    non_covid_ids = [] 

    for l in lines: 
        l_splits = l.split(',') 
        #print(l_splits)
        
        #change covid class label to 1 
        y_true.append(int(l_splits[1]))    
        
        prob0 = float(l_splits[3])
        prob1 = float(l_splits[4])

        score = np.exp(prob0)/(np.exp(prob0)+np.exp(prob1))        
        pred2 = (score < threshold).astype(np.uint8)
        
        #y_pred.append(int(l_splits[2])) 
        y_pred.append(pred2) 
        y_prob.append(score)
        
        if pred2==0:
            covid_ids.append(l_splits[0])
        else: 
            non_covid_ids.append(l_splits[0])
            
    y_pred = np.asarray(y_pred)
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)

    F1_1 = f1_score(y_true, y_pred, average='micro')
    F1_2 = f1_score(y_true, y_pred, average='macro')
    print(F1_1,F1_2)
    
    fp = open('MLP-{}-{}-covid.csv'.format(threshold,subset),'w')    
    len_covid = len(covid_ids) 
    for i in range(len_covid): 
        if subset=="test":
            ids = covid_ids[i].split(',')[0].split('_')[-1].split('.')[0]        
        else:
            ids = covid_ids[i][13:-4]
        if i<len_covid-1:
            fp.write('{},'.format(ids))
        else:
            fp.write('{}'.format(ids))
    fp.close() 

    fp = open('MLP-{}-{}-non-covid.csv'.format(threshold,subset),'w')
    len_non_covid = len(non_covid_ids) 
    for i in range(len_non_covid):
        if subset=="test":
            ids = non_covid_ids[i].split(',')[0].split('_')[-1].split('.')[0]        
        else:
            ids = non_covid_ids[i][13:-4]

        if i<len_non_covid-1:
            fp.write('{},'.format(ids))
        else:
            fp.write('{}'.format(ids))
    fp.close() 
    
    

['features_val_non-covid_ct_scan_32.pkl,1,1,-1.788,2.226', 'features_val_non-covid_ct_scan_134.pkl,1,1,-1.252,1.683', 'features_val_non-covid_ct_scan_193.pkl,1,1,-1.790,2.228', 'features_val_non-covid_ct_scan_156.pkl,1,0,1.814,-1.348', 'features_val_covid_ct_scan_3.pkl,0,0,2.452,-2.020', 'features_val_covid_ct_scan_68.pkl,0,0,2.445,-2.013', 'features_val_non-covid_ct_scan_55.pkl,1,1,-1.794,2.232', 'features_val_non-covid_ct_scan_50.pkl,1,1,-1.165,1.580', 'features_val_non-covid_ct_scan_173.pkl,1,0,0.379,0.073', 'features_val_non-covid_ct_scan_84.pkl,1,1,-1.770,2.208']
0.9197860962566845 0.9185540069686411
['features_test_subset7_a442e0ea-acf6-450f-8476-50a6b0aab1e8.pkl,0,0,2.489,-2.058', 'features_test_subset1_3d1265e7-cd86-4c77-bbf4-4bb24a3223dc.pkl,0,1,-1.780,2.218', 'features_test_subset8_cf37ec9e-98e9-4bc2-aa68-248edc41ddd6.pkl,0,1,-1.726,2.165', 'features_test_subset6_28318d31-b51c-4e22-bd8f-c3be69640ffd.pkl,0,1,-1.781,2.219', 'features_test_subset3_9d3e899d-fbc7-4868-a354-69f55af